## 각 모델별 parameter를 지정

In [44]:
import pandas as pd
import numpy as np
import json
import pickle

### Bag of Word

In [45]:
model = "bow"

num_author = [100, 200, 500, 1000]
num_title = [100, 200, 500, 1000]
num_text = [500, 1000, 2000, 5000]

# num_author = [100]
# num_title = [600]
# num_text = [1000]

### LDA

In [30]:
model = "lda"

num_author = [100, 200, 500, 1000]
num_title = [100, 200, 500, 1000]
num_text = [200, 500, 1000, 2000]

### Word2Vec

In [9]:
model = "word2vec"

num_author = [100, 200, 300, 500]
num_title = [100, 200, 300, 500]
num_text = [100, 200, 500, 1000]

## parameter별로 파일을 읽어옴

In [46]:
author_forum_p = []
title_p = []
text_p = []

for num in num_author:
    author_forum_p.append(pickle.load(open("%s/author, forum_%d.p" % (model, num), "rb")))
for num in num_title:
    title_p.append(pickle.load(open("%s/title_%d.p" % (model, num), "rb")))
for num in num_text:
    text_p.append(pickle.load(open("%s/text_%d.p" % (model, num), "rb")))

In [47]:
train = []
for author_cnt, author in enumerate(num_author):
    train.append([])
    for title_cnt, title in enumerate(num_title):
        train[author_cnt].append([])
        for text_cnt, text in enumerate(num_text):
            train[author_cnt][title_cnt].append(pd.concat([author_forum_p[author_cnt], title_p[title_cnt], text_p[text_cnt]], axis = 1))

## Train Score와 Cross Validation Score 측정

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score

label = 'istroll'

model = RandomForestClassifier(n_estimators = 10, n_jobs = -1)

In [70]:
train_score = []
cv_score = []
for author_cnt, author in enumerate(num_author):
    train_score.append([])
    cv_score.append([])
    for title_cnt, title in enumerate(num_title):
        train_score[author_cnt].append([])
        cv_score[author_cnt].append([])
        for text_cnt, text in enumerate(num_text):
            predictors = train[author_cnt][title_cnt][text_cnt].columns.drop([label])
            model.fit(train[author_cnt][title_cnt][text_cnt][predictors], train[author_cnt][title_cnt][text_cnt][label])
            predicted = model.predict_proba(train[author_cnt][title_cnt][text_cnt][predictors])
            train_score[author_cnt][title_cnt].append(roc_auc_score(train[author_cnt][title_cnt][text_cnt][label], predicted.T[1]))
            cv_score[author_cnt][title_cnt].append(np.mean(cross_val_score(model, train[author_cnt][title_cnt][text_cnt][predictors], train[author_cnt][title_cnt][text_cnt][label], scoring='roc_auc', cv=3, n_jobs=-1)))

## Test Space

In [72]:
print train_score
print cv_score

[[[0.99943534726143413]]]
[[[0.66981684981684975]]]
